In [1]:
from models import WeatherService
from models import Order
from models import DriverManager

import pandas as pd
import numpy as np
import os
import joblib
import random

import gymnasium as gym
from gymnasium import spaces

from imblearn.ensemble import BalancedRandomForestClassifier

In [2]:
class DeliveryEnv(gym.Env):

    def __init__(
        self,
        order_data: pd.DataFrame,
        driver_data: pd.DataFrame,
        order_driver_data: pd.DataFrame,
        schedule_data: pd.DataFrame,
        acceptance_model: BalancedRandomForestClassifier,
        weather_service: WeatherService,
    ):
        super(DeliveryEnv, self).__init__()

        self.order_data = order_data
        self.driver_data = driver_data
        self.order_driver_data = order_driver_data
        self.schedule_data = schedule_data
        self.acceptance_model = acceptance_model
        self.weather_service = weather_service

        self.driver_manager = DriverManager(
            order_driver_data=self.order_driver_data,
            driver_data=self.driver_data,
            schedule_data=self.schedule_data,
            acceptance_model=self.acceptance_model,
        )

        self.order_data_specific_day_concatenated = (
            self._get_order_data_specific_day_concatenated()
        )
        self.order_ids = (
            self.order_data_specific_day_concatenated["order_id"].unique().tolist()
        )
        # important info to judge if need to update driver_manager.update_driver_set
        # when change to a new day
        self.current_order_index: int = 0
        # self.current_order_date = None
        self.previous_order_date = None
        self.order_length = len(self.order_ids)
        # Define action space (continuous commission rate between 0 and 1)
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)

        # Define state space (order + specific driver attributes)
        self.observation_space = spaces.Dict(
            {
                "customer_price": spaces.Box(
                    low=0.0, high=1.0, shape=(1,), dtype=np.float32
                ),
                "pickup_area": spaces.Discrete(501),
                "dropoff_area": spaces.Discrete(501),
                "hour_of_day": spaces.Discrete(24),
                "day_of_week": spaces.Discrete(7),
                "weather": spaces.Discrete(4),
                "driver_area": spaces.Discrete(501),
                "working_status": spaces.Discrete(2),
            }
        )

        self.state = None
        self.steps = 0

    def reset(self, seed=None):
        super().reset(seed=seed)

        self.driver_manager = DriverManager(
            order_driver_data=self.order_driver_data,
            driver_data=self.driver_data,
            schedule_data=self.schedule_data,
            acceptance_model=self.acceptance_model,
        )
        self.order_data_specific_day_concatenated = (
            self._get_order_data_specific_day_concatenated()
        )
        self.order_ids = (
            self.order_data_specific_day_concatenated["order_id"].unique().tolist()
        )
        # important info to judge if need to update driver_manager.update_driver_set
        # when change to a new day
        self.current_order_index: int = 0
        # self.current_order_date = None
        self.previous_order_date = None

        self.state = None
        return self.state, {}

    def step(self, action: float):

        current_order_id = self.order_ids[self.current_order_index]
        order_data_specific_day_concatenated: pd.DataFrame = (
            self.order_data_specific_day_concatenated
        )

        order_info = order_data_specific_day_concatenated[
            order_data_specific_day_concatenated["order_id"] == current_order_id
        ]
        order_info: pd.Series = order_info.iloc[0]
        current_order_datetime = order_info["date"]

        if current_order_datetime != self.previous_order_date:
            print(
                f"The date of Current Order {current_order_datetime} is different from Previous Order {self.previous_order_date}"
            )
            print("Need to set update driver set to empty")
            self.driver_manager.update_driver_set = (
                self.driver_manager.update_driver_set.head(0)
            )

        # ---------------------------------------------------
        order = Order(
            order_id=order_info["order_id"],
            datetime_str=order_info["datetime"],
            pickup_area=order_info["pickup_area2"],
            dropoff_area=order_info["dropoff_area2"],
            pickup_lat=order_info["pickup_lat"],
            pickup_lon=order_info["pickup_lon"],
            dropoff_lat=order_info["dropoff_lat"],
            dropoff_lon=order_info["dropoff_lon"],
            customer_price=order_info["customer_price"],
            commissionPercent=action,
            complete_time=order_info["complete_time"],
            weather_service=self.weather_service,
        )
        print(order)

        # ---------------------------------------------------
        accept_order = self.driver_manager.get_driver_attampt(order=order)
        if accept_order:
            reward = order.platform_revenue
        else:
            reward = 0
        # ---------------------------------------------------
        # update current order index for the next order
        self.current_order_index += 1
        self.previous_order_date = current_order_datetime
        print(f"Update current_order_index to {self.current_order_index}")

        # ---------------------------------------------------
        # check the termination condition
        if self.current_order_index >= self.order_length:
            terminated = True
            print("Reach terminated condition: Finish simulating all the orders.")
        else:
            terminated = False

        # ---------------------------------------------------
        # no truncated condition
        truncated = False

        return self.state, reward, terminated, truncated, {}

    def render(self, mode="human"):
        print(f"Step: {self.steps}, State: {self.state}")

    def close(self):
        pass

    def _get_order_data_specific_day_concatenated(self):
        order_data_columns = [
            "order_id",
            "datetime",
            "date",
            "pickup_area2",
            "dropoff_area2",
            "pickup_lat",
            "pickup_lon",
            "dropoff_lat",
            "dropoff_lon",
            "customer_price",
            "complete_time",
        ]
        valid_days = self.order_data["date"].unique().tolist()
        random.shuffle(valid_days)
        # Initialize an empty list to store processed dataframes
        all_processed_days_data = []

        for day in valid_days:  # Loop through each valid day
            order_data_specific_day = self.order_data[
                self.order_data["date"] == day
            ]  # Use the current day from the loop
            order_data_specific_day = order_data_specific_day.dropna(
                subset=order_data_columns
            )
            order_data_specific_day = order_data_specific_day.sort_values(
                "datetime", ascending=True
            )
            order_data_specific_day = order_data_specific_day[order_data_columns]
            all_processed_days_data.append(
                order_data_specific_day
            )  # Append the processed dataframe for the current day
        # Concatenate all dataframes in the list
        order_data_specific_day_concatenated = pd.concat(all_processed_days_data)

        return order_data_specific_day_concatenated

---

## Test Delivery Enviornment

In [3]:
schedule_data = pd.read_csv("./data/driver_schedule.csv", engine="pyarrow")
driver_data = pd.read_csv("./data/driver_update2.csv", engine="pyarrow")
order_data = pd.read_csv("./data/order.csv", engine="pyarrow")
order_driver_data = pd.read_csv("./data/order_driver.csv", engine="pyarrow")
weather_service = WeatherService(weather_csv_path="./data/weather.csv")

In [4]:
model_output_folder = "./out"
os.makedirs(model_output_folder, exist_ok=True)
model_path = "./Out/acceptance_model.pkl"

# if the model exists, decide if re-train the model is needed
retrain_model: bool = False

if os.path.exists(model_path) and not retrain_model:
    print(f"Model found at {model_path}. Loading model...")
    # with open(model_path, "rb") as f:
    #     model = pickle.load(f)
    model = joblib.load(model_path)
    print("Model loaded successfully.")

Model found at ./Out/acceptance_model.pkl. Loading model...
Model loaded successfully.


In [5]:
env = DeliveryEnv(
    order_data=order_data,
    driver_data=driver_data,
    order_driver_data=order_driver_data,
    schedule_data=schedule_data,
    acceptance_model=model,
    weather_service=weather_service,
)

In [16]:
env.step(0.2)

Order(
    order_id=4877449,
    datetime=2025-04-15 08:08:14,
    pickup_area=403.0,
    dropoff_area=407.0,
    pickup_lat=32.6521337,
    pickup_lon=51.6843834,
    dropoff_lat=32.6483004,
    dropoff_lon=51.7152642,
    customer_price=58000.00,
    commissionPercent=0.20,
    driver_commission=11600.00,
    platform_revenue=46400.00,
    hour_of_day=8
    weather_code=1.0
    complete_time=101.33333333333333
)
No new drivers from update_driver_set with matching 'driver_area' ('403.0') to add.
update_driver_set has no matched driver ID for existing drivers in the pool, no update from it.
Driver pool has been randomized.
Driver 15166.0 is initialized with location (32.6419666, 51.6550004)
Driver 15166.0 can work at 2025-04-15 08:00.
The distance calculated by geodesic is 2978.476773637889
Features input to the model for prediction:
{'commission': [np.float64(11600.0)], 'driver_distance': [2978.476773637889], 'hour': [8], 'weather_code': [1.0], 'work_time_minutes': [np.float64(5.73333

(None, np.float64(46400.0), False, False, {})

In [17]:
env.driver_manager.update_driver_set

,driver_id,driver_lat,driver_lon,driver_area,work_time_minutes
0,20799.0,32.650019,51.675121,208.0,46.983333
1,3048.0,32.588546,51.670628,400.0,19.833333
2,8847.0,32.643459,51.554933,405.0,45.383333
3,21663.0,32.654958,51.675185,330.0,14.483333
4,15166.0,32.648300,51.715264,403.0,107.066667
